In [ ]:
erkläre mir schriit für schritt, wie diese code es schafft als HSI RGB bilder zuerzeugen. schreibe alle formel die verwenden werde, falls welche ghibt und erkläre jeden komponente der formel und wie das funktioniert. erkläre alles was hier gemacht wird bis es zu RGB bild wird. hier ist der code:
# spectral_tools.py

import numpy as np
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d
from colour import MSDS_CMFS, SDS_ILLUMINANTS, XYZ_to_sRGB
import os
import scipy.io
import matplotlib.pyplot as plt

### 🔁 Superresolution: mehr Kanäle erzeugen durch Interpolation
def enhance_spectral_resolution(hcube, target_bands=200):
    """Interpoliert spektrale Achse auf Zielanzahl Bänder."""
    H, W, B = hcube.shape
    hcube_interp = np.zeros((H, W, target_bands), dtype=np.float32)
    x_old = np.linspace(0, 1, B)
    x_new = np.linspace(0, 1, target_bands)

    for i in range(H):
        for j in range(W):
            f = interp1d(x_old, hcube[i, j, :], kind='cubic', fill_value='extrapolate')
            hcube_interp[i, j, :] = f(x_new)
    return hcube_interp


### 🎯 Neutrales Referenzfeld finden
def find_neutral_patch(hcube):
    """Findet Position mit geringster spektraler Varianz (graue Fläche)."""
    spectral_std = np.std(hcube, axis=2)
    y, x = np.unravel_index(np.argmin(spectral_std), spectral_std.shape)
    return x, y


### ✅ Kalibrierung prüfen auf Basis Spektralon-Patch
def check_calibration(hcube, ref_x, ref_y, tol=0.1):
    """Warnung, wenn der Referenzbereich nicht ca. 0.95 +/- tol ist."""
    patch = hcube[ref_y-2:ref_y+3, ref_x-2:ref_x+3, :]
    patch_mean = np.mean(patch, axis=(0, 1))
    below = patch_mean < (0.95 - tol)
    above = patch_mean > (0.95 + tol)

    if np.any(below) or np.any(above):
        print("[⚠️] Warnung: Weißreferenz außerhalb Toleranzbereich!")
        print("    Min Abw.:", np.min(patch_mean))
        print("    Max Abw.:", np.max(patch_mean))
    else:
        print("[✅] Weißreferenz innerhalb tolerierbarer Grenzen")
    return patch_mean


### 🔍 Wellenlängenschätzung aus mittlerem Spektrum
def estimate_wavelengths(hcube):
    """Schätzt spektrale Wellenlängenpositionen aus dem mittleren Verlauf."""
    mean_spectrum = np.mean(hcube, axis=(0, 1))
    smoothed = savgol_filter(mean_spectrum, 11, 3)
    estimated = np.linspace(400, 700, len(mean_spectrum))
    peak_shift = np.argmax(smoothed[:len(mean_spectrum)//3]) - 50
    return np.clip(estimated - peak_shift, 380, 780)


### 🌈 Hyperspektral zu RGB mit neuer Modularstruktur
def convert_hsi_to_rgb(mat_path, output_path, use_superres=False):
    mat = scipy.io.loadmat(mat_path)
    if "ref" not in mat:
        raise ValueError("MAT-Datei enthält kein 'ref'-Feld")

    hcube = mat["ref"].astype(np.float32)
    hcube = np.clip(hcube, 0, None)
    hcube = hcube / np.percentile(hcube, 99.99)

    if use_superres:
        print("[🔁] Spektrale Superresolution wird angewendet...")
        hcube = enhance_spectral_resolution(hcube, target_bands=200)

    wavelengths = estimate_wavelengths(hcube)
    visible_mask = (wavelengths >= 380) & (wavelengths <= 780)
    hcube_vis = hcube[:, :, visible_mask]
    wavelengths_vis = wavelengths[visible_mask]

    ref_x, ref_y = find_neutral_patch(hcube_vis)
    print(f"[🎯] Neutraler Patch gefunden bei: ({ref_x}, {ref_y})")
    check_calibration(hcube_vis, ref_x, ref_y)

    patch = hcube_vis[ref_y-2:ref_y+3, ref_x-2:ref_x+3, :]
    white_ref = np.mean(patch, axis=(0, 1))
    hcube_norm = hcube_vis / (white_ref + 1e-12)

    cmfs = MSDS_CMFS['CIE 1931 2 Degree Standard Observer']
    illuminant = SDS_ILLUMINANTS['D65']
    wl_cmfs = cmfs.wavelengths
    valid_cmfs = (wl_cmfs >= 380) & (wl_cmfs <= 780)

    xyz_weights = np.array([
        np.interp(wavelengths_vis, wl_cmfs[valid_cmfs], cmfs.values[valid_cmfs, i])
        for i in range(3)
    ])
    ill_interp = np.interp(wavelengths_vis, illuminant.wavelengths, illuminant.values)
    xyz_weights *= ill_interp

    xyz = np.tensordot(hcube_norm, xyz_weights.T, axes=([2], [0]))
    xyz = xyz / (xyz[..., 1].max() + 1e-8)
    rgb = XYZ_to_sRGB(xyz)
    rgb = np.clip(np.sqrt(rgb), 0, 1)

    os.makedirs(output_path, exist_ok=True)
    filename = os.path.splitext(os.path.basename(mat_path))[0]
    save_path = os.path.join(output_path, f"{filename}_rgb_new.png")
    plt.imsave(save_path, rgb)
    print(f"[✅] RGB gespeichert unter: {save_path}")


if __name__ == "__main__":
    convert_hsi_to_rgb(
    mat_path="./data/harvard/imge3.mat",
    output_path="./outputs/tesla_rgb",
    use_superres=True  # oder False, je nach Wunsch
)